<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Geographic Arbitrage

_by Joe Hamman (CarbonPlan) and Grayson Badgley, January 25, 2021_


In [ ]:
!pip install descartes
!pip install -e ..

In [ ]:
import geopandas
import matplotlib.pyplot as plt
from shapely.geometry import Point
from sklearn.neighbors import KDTree
import numpy as np
import pandas as pd

from carbonplan import styles
from carbonplan.data import cat as core_cat
from carbonplan_retro.data import cat
from carbonplan_retro.load.fia import load_fia_state_long, to_geodataframe
from carbonplan_retro.load.arb_fortypcds import load_arb_fortypcds

styles.mpl.set_theme(style="carbonplan_light")
colors = styles.colors.light

## Input data

#### 4km Albers Equal Area Grid


In [ ]:
ds = core_cat.grids.conus4k.to_dask()

display(ds)

mask = ds.mask.values.flat > 0
crs = ds.crs.attrs["crs_wkt"]

#### FIA Condition Level Table


In [ ]:
df_ca = load_fia_state_long("ca")
df_or = load_fia_state_long("or")
df = pd.concat([df_ca, df_or], axis=0, ignore_index=True)
df = df[(df["MEASYEAR"] > 2000) & (df["MEASYEAR"] < 2015)]
df = df[(df["OWNCD"] == 46)]

# project to Albers
df = df.to_crs(crs)

display(df.head())

#### Supersection geometries


In [ ]:
supersections = cat.supersections.read().set_index("SSection").to_crs(crs=crs)
section_name = "Northern California Coast"  # "Southern Cascades"  # we'll use this for testing
supersections.loc[[section_name]].plot()

### Create the evaluation mesh


In [ ]:
# create the mesh that we will sample from
points = [Point(x, y) for x, y in zip(ds.lon.values.flat[mask], ds.lat.values.flat[mask])]

mesh = geopandas.GeoDataFrame(
    data={"num": list(range(len(points)))}, crs="epsg:4326", geometry=points
)
mesh = mesh.to_crs(crs=crs)
mesh.plot(markersize=0.01)

In [ ]:
# show that we can subset the mesh by supersection
supersection_mesh = geopandas.clip(mesh, supersections.loc[[section_name]])
supersection_mesh.plot(markersize=0.1)

Next we want to calculate the average above ground biomass for each mesh location based on its
nearest 100 neighbors. For this example, we look at only FORTYPCD < 400. We use a `KDTree` to
quickly select the closest points.


In [ ]:
# get a subset of the FIA table
subset_df = df[df["FORTYPCD"] < 400]

# create the KDTree
tree = KDTree(np.stack([subset_df.geometry.x.values, subset_df.geometry.y.values], axis=1))

# pick a sample point in the mesh
pind = 300
p = supersection_mesh.geometry.iloc[pind]
xy = (p.x, p.y)

# Find the indicies of the nearest 100 points
inds = tree.query([xy], k=100, return_distance=False)[0]

# Finally, pull out the 100 conds from the FIA table
p_plots = subset_df.iloc[inds]

In [ ]:
p_plots[p_plots["FORTYPCD"] == 201]["slag_co2e_acre"].plot.hist()

A quick visual. Belovalue_counts the full supersection mesh (grey), a single evaluation point
(black), and the selected FIA Conditions (red).


In [ ]:
fig, ax = plt.subplots(figsize=(5, 10))

supersection_mesh.plot(ax=ax, c=colors["secondary"], markersize=1, label="sample mesh")
p_plots.plot(ax=ax, c=colors["red"], markersize=2, label="fia points")
supersection_mesh.iloc[[pind]].plot(ax=ax, c=colors["primary"], label="sample point")

# ax.legend()

#### The simplest arbitrage map

Now, we can calculate a simple version of the arbitrage map by calculating the mean above ground
biomass for all mesh locations by taking the average of the 100 closest FIA conditions.


In [ ]:
mean_agb = []
for p in supersection_mesh.geometry:
    xy = p.x, p.y
    inds = tree.query([xy], k=100, return_distance=False)[0]
    mean_agb.append(subset_df["adj_ag_biomass"].iloc[inds].mean())
supersection_mesh["mean_agb"] = mean_agb

In [ ]:
supersection_df = geopandas.clip(subset_df.reindex(), supersections.loc[[section_name]])
supersection_mesh["mean_agb_diff"] = (
    supersection_mesh["mean_agb"] - supersection_df["adj_ag_biomass"].mean()
)

Now we plot the above ground biomass map.


In [ ]:
fig, ax = plt.subplots(figsize=(5, 10))
ax = supersection_mesh.plot("mean_agb", cmap="Greens", legend=True, ax=ax, markersize=3)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 10))
ax = supersection_mesh.plot(
    "mean_agb_diff", cmap="PRGn", legend=True, ax=ax, markersize=3, vmin=-150, vmax=150
)

### all together now!

The cell below is semi-complete but should illustrate how to repeate the steps above for all
supersections/assessment areas.


In [ ]:
supersection_names = ["Southern Cascades", "Northern California Coast"]
assessment_areas = [297, 221]

fortype_dict = load_arb_fortypcds()

results = {}
subsets = {}

for supersection_name, aa_id in zip(supersection_names, assessment_areas):
    print(supersection_name, aa_id)

    subset_df = df[df["FORTYPCD"].isin(fortype_dict[aa_id])]

    tree = KDTree(np.stack([subset_df.geometry.x.values, subset_df.geometry.y.values], axis=1))

    supersection_mesh = geopandas.clip(mesh, supersections.loc[[supersection_name]])

    mean_agb = []
    for p in supersection_mesh.geometry:
        xy = p.x, p.y
        inds = tree.query([xy], k=100, return_distance=False)[0]
        mean_agb.append(subset_df["adj_ag_biomass"].iloc[inds].mean())
    supersection_mesh["mean_agb"] = mean_agb

    results[supersection_name] = supersection_mesh
    subsets[supersection_name] = subset_df

In [ ]:
for supersection_name in supersection_names:

    supersection_mesh = results[supersection_name]
    subset_df = subsets[supersection_name]

    supersection_df = geopandas.clip(subset_df.reindex(), supersections.loc[[supersection_name]])
    supersection_mesh["mean_agb_diff"] = (
        supersection_mesh["mean_agb"] - supersection_df["adj_ag_biomass"].mean()
    )

    fig, ax = plt.subplots(figsize=(5, 10))
    ax = supersection_mesh.plot("mean_agb", cmap="Greens", legend=True, ax=ax, markersize=3)

    fig, ax = plt.subplots(figsize=(5, 10))
    ax = supersection_mesh.plot(
        "mean_agb_diff", cmap="PRGn", legend=True, ax=ax, markersize=3, vmin=-100, vmax=100
    )